<img src="https://images.efollett.com/htmlroot/images/templates/storeLogos/CA/864.gif" style="float: right;"> 




# ECON628-01 
### Lecture 1.2 - Principal Component Analysis PCA
---

# PCA and Horn's Parallel Analysis Lab

In this lab you'll practice using PCA on the heptathalon performance data set
Horn's Parallel Analysis is a way to determine how many components you should keep after using a PCA on your data. Essentially it will tell you which of your components are likely noise which can be discarded.

---

### Load packages and heptathalon data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use('fivethirtyeight')

from ipywidgets import *
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
hep = pd.read_csv('heptathlon.csv')

In [ ]:
hep.columns[1:]

In [ ]:
hep.columns = ['athlete'] + hep.columns[1:].tolist()

In [ ]:
hep.head(3)

In [ ]:
hep.iloc[:,1:].corr()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
hep_n = StandardScaler().fit_transform(hep.iloc[:,1:-1])
hep_n[:,[0,3,6]] *= -1


In [ ]:
pca = PCA().fit(hep_n)

In [ ]:
pca.components_

In [ ]:
pca1_evec = pca.components_[0]
for weight, event in zip(pca1_evec, hep.iloc[:,1:-1].columns):
    print event, weight

In [ ]:
pca2_evec = pca.components_[2]
for weight, event in zip(pca2_evec, hep.iloc[:,1:-1].columns):
    print event, weight

---

### Create dataframe excluding athlete and score

---

### Examine the correlation between the different events

Plot a heatmap if you want to get fancy. What does the correlation matrix tell you?

---

### Standardize the data

In [ ]:
from sklearn.preprocessing import StandardScaler




---

### Fit a PCA on the standardized data using sklearn

In [ ]:
hep_pca = PCA()
hep_pca.fit(??)
stats_pcs = hep_pca.transform(??)

---

### Create a DataFrame with the principal components

Add back in the athelete and score columns from the original data.

In [ ]:
stats_pcs = pd.DataFrame(stats_pcs, columns=['PC'+str(i) for i in range(1,8)])
stats_pcs['??'] = hep.iloc[:,0]
stats_pcs['??'] = hep.score

In [ ]:
hep_pca.components_

---

### Plot the variance explained (ratio) of your components

Explain what this chart tells you about your components.

In [ ]:
hep_pca.explained_............????

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(range(1, stats.shape[1]+1), hep_pca.explained_variance_ratio_, lw=2)
ax.scatter(range(1, stats.shape[1]+1), hep_pca.explained_variance_ratio_, s=100)
ax.set_title('explained variance of components')
ax.set_xlabel('principal component')
ax.set_ylabel('explained variance')
plt.show()

---

### Print out the weights/eigenvectors (.components_ ) with their corresponding variables for PC1 and PC2

Based on how the original variables are weighted to calculate the components, how would you describe PC1 and PC2?

In [ ]:
for col, comp in zip(stats.columns, hep_pca.components_[??]):
    print col, comp

In [ ]:
for col, comp in zip(stats.columns, hep_pca.components_[??]):
    print col, comp

In [ ]:
stats_pcs.head()

---

### Plot PC1 vs. PC2. Which athletes are notable on each component?

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(stats_pcs.PC1.values, stats_pcs.PC2.values, s=0)

for i, txt in enumerate(stats_pcs.athlete.values):
    ax.annotate(txt, (0, 0), (stats_pcs.PC1.values[i], stats_pcs.PC2.values[i]),
            arrowprops=dict(arrowstyle='<-', color='black', linewidth=1.5),
            xycoords='data', textcoords='data', fontsize=12, color="black")

ax.set_title('PC1 (run) vs. PC2 (javelin/shot)')
ax.set_xlabel('principal component 1 (run)')
ax.set_ylabel('principal component 2 (javelin/shot)')
plt.show()

---

### Plot PC1 vs. score and PC2 vs. score. What does this tell you about the relationship between the events and the score?

In [ ]:
fig, ax = plt.subplots(figsize=(8,7))
ax.scatter(stats_pcs.???.????, stats_pcs.???.??, s=100)

ax.set_title('PC1 (run) vs. score')
ax.set_xlabel('principal component 1 (run)')
ax.set_ylabel('score')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,7))
ax.scatter(??, ???, s=100, c='darkred')

ax.set_title('PC2 (shot/javelin) vs. score')
ax.set_xlabel('principal component 2 (shot/javelin)')
ax.set_ylabel('score')
plt.show()

---

### Horn's parallel analysis

You can determine the appropriate number of components to keep by using a bootstrapping procedure known as Horn's Parallel Analysis. This is (as far as I know) the gold standard in determining which components aren't noise.

How to do the parallel analysis (pseudocode):

    for n iterations:
        create normally distributed random data the same shape as your data
        fit a PCA on the random data
        pull out the eigenvalues
    select a percentile of the eigenvalues as your threshold (0.5 = median, 0.95 = 95% confidence, etc.)
    plot the random component eigenvalues at that percentile against your data's pca eigenvalues
    components above the selected percentile are not noise, those under are
    
    
Write a function to perform the parallel analysis.

In [ ]:
def horn_parallel_analysis(shape, iters=1000, percentile=95):
    pca = PCA(n_components=shape[1])
    eigenvals = []
    for i in range(iters):
        rdata = np.random.normal(0,1,size=shape)
        pca.fit(rdata)
        eigenvals.append(pca.explained_variance_)
    eigenvals = np.array(eigenvals)
    return np.percentile(eigenvals, percentile, axis=0)

---

### Run parallel analysis for the data

In [ ]:
hep_pa = horn_parallel_analysis(hep.iloc[:,1:-1].shape, percentile=95)

---

### Plot the wine eigenvalues (`.variance_explained_`) against the parallel analysis random eigenvalue cutoffs

How many components are not noise, based on the chart?

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax.plot(range(1, hep.iloc[:,1:-1].shape[1]+1), hep_pca.explained_variance_, lw=2)
ax.scatter(range(1, hep.iloc[:,1:-1].shape[1]+1), hep_pca.explained_variance_, s=50)

ax.plot(range(1, len(hep_pa)+1), hep_pa, lw=2, color='darkred')
ax.scatter(range(1, len(hep_pa)+1), hep_pa, s=40, color='darkred')


ax.set_title('Horns parallel analysis on hep data components')
ax.set_xlabel('principal component')
ax.set_ylabel('eigenvalue')
plt.show()